In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# -*- coding: utf-8 -*-
import argparse
import math
import sys
import time
import copy
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization#, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
#K.set_image_dim_ordering('th')
#print(K.image_data_format())

## required for efficient GPU use
#import tensorflow as tf
#from keras.backend import tensorflow_backend
#config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
#session = tf.Session(config=config)
#tensorflow_backend.set_session(session)
## required for efficient GPU use

import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

In [6]:
model_path='/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/Shallow CNN Model/results/cnn_shallow_mawi3.h5'

# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

In [7]:
# get data

class dataset:
    mawi_train_2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_train_2labels(001).pkl")
    mawi_test_2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_test_2labels(004).pkl")
    
    
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

class preprocess:
    
    output_columns_2labels = ['status_anomaly','status_normal']
    
    x_input = dataset.mawi_train_2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.mawi_train_2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.mawi_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.mawi_test_2labels.loc[:,output_columns_2labels]
    
    ss = pp.StandardScaler()
    
    x_train = ss.fit_transform(x_input)
    x_test = ss.fit_transform(x_test_input)
    
    y_train = y_output.values
    y_test = y_test.values
        
    print('x_train shape: {}'.format(x_train.shape))
    print('y_train shape: {}'.format(y_train.shape))
    print('x_test shape: {}'.format(x_test.shape))
    print('y_test shape: {}'.format(y_test.shape))

x_train shape: (40934, 4)
y_train shape: (40934, 2)
x_test shape: (42599, 4)
y_test shape: (42599, 2)


In [8]:
#reshape data
X_train = np.reshape(preprocess.x_train, (preprocess.x_train.shape[0], preprocess.x_train.shape[1], 1))
X_test = np.reshape(preprocess.x_test, (preprocess.x_test.shape[0], preprocess.x_test.shape[1], 1))

print('train shape after reshape: {}'.format(X_train.shape))
print('test shape after reshape: {}'.format(X_test.shape))

train shape after reshape: (40934, 4, 1)
test shape after reshape: (42599, 4, 1)


In [9]:
#size of parameters
batch_size = 64
num_classes = 2
epochs = 10
filter_size=3
#noise = 1
droprate=0.50

In [10]:
#Start Neural Network
model = Sequential()

#convolution 1st layer
model.add(Conv1D(64, kernel_size=(filter_size), padding="same",
                 activation='relu',
                 input_shape=(4, 1)))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(MaxPooling1D(strides=1))

#FCN layer
model.add(Flatten())
model.add(Dense(64,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 4, 64)             256       
                                                                 
 batch_normalization (BatchN  (None, 4, 64)            256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 4, 64)             0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 3, 64)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 192)               0         
                                                  

In [11]:
#Save Model=ON
history = model.fit(X_train, preprocess.y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, preprocess.y_test),shuffle=True,callbacks=callbacks)

score = model.evaluate(X_test, preprocess.y_test, verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 40934 samples, validate on 42599 samples
Epoch 1/10
39936/40934 [============================>.] - ETA: 0s - loss: 0.4292 - acc: 0.8097

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


40934/40934 [==============================] - 3s 80us/sample - loss: 0.4280 - acc: 0.8102 - val_loss: 0.5044 - val_acc: 0.7069
Epoch 2/10
40934/40934 [==============================] - 2s 60us/sample - loss: 0.3731 - acc: 0.8301 - val_loss: 0.4840 - val_acc: 0.7062
Epoch 3/10
40934/40934 [==============================] - 3s 61us/sample - loss: 0.3670 - acc: 0.8316 - val_loss: 0.5054 - val_acc: 0.7089
Epoch 4/10
40934/40934 [==============================] - 2s 61us/sample - loss: 0.3617 - acc: 0.8336 - val_loss: 0.4735 - val_acc: 0.7083
Epoch 5/10
40934/40934 [==============================] - 2s 61us/sample - loss: 0.3583 - acc: 0.8336 - val_loss: 0.4605 - val_acc: 0.7093
Epoch 6/10
40934/40934 [==============================] - 2s 61us/sample - loss: 0.3563 - acc: 0.8339 - val_loss: 0.4635 - val_acc: 0.7097
Epoch 7/10
40934/40934 [==============================] - 2s 60us/sample - loss: 0.3556 - acc: 0.8356 - val_loss: 0.4982 - val_acc: 0.7093
Epoch 8/10
40934/40934 [==============

In [12]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

predict_proba = model.predict(X_test)
predict_classes = np.argmax(predict_proba, axis=1)
#y_pred = model.predict_classes(X_test)
#print(y_pred)

#p=model.predict_proba(X_test)

target_names = ['normal', 'anomaly']
print(classification_report(np.argmax(preprocess.y_test, axis=1), y_pred, target_names=target_names, digits=4))
print(confusion_matrix(np.argmax(preprocess.y_test, axis=1), y_pred))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[1.6558096e-01 8.3441901e-01]
 [1.7052606e-02 9.8294741e-01]
 [1.5658203e-01 8.4341794e-01]
 ...
 [6.1045590e-08 9.9999988e-01]
 [6.1045590e-08 9.9999988e-01]
 [1.6732490e-01 8.3267510e-01]]
[1 1 1 ... 1 1 1]
              precision    recall  f1-score   support

      normal     0.7722    0.0715    0.1308     13040
     anomaly     0.7075    0.9907    0.8255     29559

    accuracy                         0.7093     42599
   macro avg     0.7398    0.5311    0.4782     42599
weighted avg     0.7273    0.7093    0.6128     42599

[[  932 12108]
 [  275 29284]]
